<a href="https://colab.research.google.com/github/JoaquinV13/UFV-Visualizacion/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly_express as px
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'plotly_express'